In [1]:
from artist import Artist, Song
import pickle
from glob import glob
from pathlib import Path
import os
from pyspark import SparkContext
from collections import Counter, defaultdict

sc = SparkContext()
data_folder = "./data"
artists = []
for file in glob(data_folder + "/*.pickle"):
    with open(file, "rb") as f:
        artist = pickle.load(f)
    if len(artist.songs) > 0 :
        artists.append(artist)
    else:
        os.remove(file)

# orel = []
# for writer in writers:
#     if 'Orel' in writer.name:
#         orel.append(writer)
# orel
print([len(art.songs )for art in artists])

[33, 82, 93, 38, 42, 47, 260, 13, 56, 53, 22, 43, 207, 100, 49, 33, 38, 87, 35, 143, 28, 4, 76, 159, 78, 269, 36, 228, 283, 64, 40, 36, 42, 54, 64, 124, 95, 33, 34, 35, 25, 4, 162, 48, 48, 50, 1, 51, 30, 29, 47, 97, 104, 102, 41, 50, 70, 283, 44, 47, 45, 33, 51, 35, 42, 69, 155, 67, 3, 122, 72, 50, 33, 14, 55, 37, 26, 57, 62, 107, 6, 108, 20, 21, 83, 123, 44, 100, 24, 67, 43, 170, 68, 296, 99, 22, 99, 28, 29, 146, 265, 122, 10, 2, 30, 81, 39, 4, 174, 3, 22, 33, 84, 74, 4, 37, 60, 14, 18, 31, 15, 135, 70, 74, 33, 13, 55, 51, 62, 90, 7, 100, 39, 15, 98, 13, 43, 20, 19, 39, 33, 129, 65, 220, 26, 101, 143, 128, 48, 207, 15, 45, 247, 27, 57, 79, 92, 146, 15, 115, 124, 4, 21, 69, 55, 35, 77, 76, 98, 332, 59, 55, 66, 40, 69, 51, 31, 198, 60, 36, 54, 54, 26, 66, 31, 134, 221, 103, 42, 16, 9, 574, 177, 37, 76, 101, 135, 90, 40, 21, 27, 53, 75, 163, 41, 131, 27, 86, 72, 45, 65, 25, 44, 6, 82, 63, 144, 20, 54, 14, 17, 359, 152, 145, 106, 139, 229, 32, 38, 35, 141, 60, 8, 90, 32, 13, 83, 61, 79, 4

In [2]:
songs = set()
all_writers = defaultdict(int)
for artist in artists:
    songs.update([s for s in  artist.songs.values() if len(s.writers) > 1])
for song in songs:
    writers = song.writers
    for writer in writers:
        all_writers[writer] += 1

# writers = sorted(list(all_writers), key= lambda x: x.name)
# writers = [artist for artist,count in list(Counter(writers).items()) if count > 1]
all_writers = {i:k for i,k in all_writers.items() if k >2}
all_writers = list(sorted(list(all_writers), key= lambda x: x.name))
len(all_writers)

966

In [3]:
from typing import Tuple
import requests as re 
from bs4 import BeautifulSoup
def get_name_from_link(link: str) -> Tuple[str,str]:
    content = re.get(link).text
    return (BeautifulSoup(content).find("h1").text.strip().replace("\n", "").split("            ")[0].split("(")[0].strip() ,link)
# print(get_name_from_link(all_writers[868].link), all_writers[868].name)

In [4]:
# artists[artists.index(Artist("https://genius.com/artists/Mb", "MB"))]

In [5]:

artist_list = sc.parallelize([w.link for w in all_writers]).map(get_name_from_link).collect()

In [6]:
all_writers = [Artist(link,name) for name, link in artist_list]
link_to_actual_artist = {artist.link:artist.name for artist in all_writers}
# link_to_actual_artist["https://genius.com/artists/Mb"] = "MB"
# link_to_actual_artist["https://genius.com/artists/Solo"] = "Solo"
# link_to_actual_artist["https://genius.com/artists/Seven"] = "Seven"
# # link_to_actual_artist.items()

In [7]:
from collections import defaultdict

# writers_dict  = {writer.name:defaultdict(lambda : defaultdict(int, weight=0)) for writer in all_writers}
writers_dict  = {writer.name:defaultdict(list) for writer in all_writers}
# print(writers_dict)
for song in songs: 
    writers = song.writers
    for writer in writers:
        if writer in all_writers:
            writer_name = link_to_actual_artist[writer.link]
            for other_writer in writers:
                if other_writer in all_writers:
                    other_name = link_to_actual_artist[other_writer.link]
                    if writer_name != other_name:
                        writers_dict[writer_name][other_name].append(song.name)
                        # writers_dict[writer_name][other_name]["weight"] +=  1
# writers_dict

In [8]:
import networkx as nx
import pyvis
# net.nodes
def get_number_of_feat(dico, artist_name,artist_bool):
    if artist_bool:
        return len(dico[artist_name])
    else:
        all_feats=  dico[artist_name]
        morceaux = set()
        for artist in all_feats:
            feat_artist = all_feats[artist]
            for morceau in feat_artist:
                morceaux.add(morceau)
        return len(morceaux)
gros_feateurs = sorted([(artist, get_number_of_feat(writers_dict,artist,False)) for artist in writers_dict], key= lambda x: x[1], reverse= True)[:50]
gros_feateurs = sorted([(artist, get_number_of_feat(writers_dict,artist,True)) for artist in writers_dict], key= lambda x: x[1], reverse= True)[:50]
gros_feateurs
# network = nx.Graph
# for artist in 
writers_dict["Swift Guad"].keys()

dict_keys(['Ol Zico', 'Jarod', "Limsa d'Aulnay", 'Paco', "Al'Tarba", 'Tragik', 'Grödash', 'JP Manova', 'Taïro', 'Geule Blansh', 'Iron Sy', '10vers', 'Furax Barbarossa', 'Jeff Le Nerf', 'Original Tonio', 'Davodka', 'Nakk Mendosa', 'Loko', 'Katana', "L'indis", 'Reeno', 'Brack', 'Ritzo', 'Char', 'Aketo', 'Sidi Omar', 'Mokless', 'Sëar Lui-Même', 'Busta Flex', 'Seyté', 'Zek', 'Fonik', 'Zesau', 'Joe Lucazz', "L'Affreux Jojo", 'Lavokato', 'Koma', 'Senamo', 'Ades', 'Hugo TSR', 'Nekfeu', 'Guizmo', 'TonyToxik', 'Cenza', 'Deen Burbigo', 'Melan', 'Nefaste', 'Mani Deïz', 'Lucio Bukowski', 'Pejmaxx', 'Titan Mc Dyess', 'Wira', 'Nasme', 'Lyricson', 'Alibi Montana', 'Dooz Kawa', 'Kacem Wapalek', 'Titan', 'Ixzo', 'Starline', 'Lacraps', 'Hyacinthe', 'Nusky', 'Demi Portion', 'Scylla', 'Monotof', "L'Hexaler", 'Sprinter', "Youssef Swatt's", 'A2H', 'OrelSan', 'Nessbeal', 'Alkpote', 'Seth Gueko', '3010', 'Noss', 'Deadi', 'R.E.D.K.', 'Faf Larage', "L'amir'Al", 'Relo', 'Boss One', 'USKY', 'Zoxea', 'Rob-D', '25G

In [9]:
from copy import deepcopy
import json
network = nx.Graph()
new_dico= deepcopy(writers_dict)
rappeurs_vus = set()
for artist_name in new_dico:
    network.add_node(artist_name,size = len(new_dico[artist_name]))


for artist_name in new_dico:
    for other_artist in new_dico[artist_name]:
        network.add_edge(artist_name,other_artist, weight = len(new_dico[artist_name][other_artist]))
connected_components = list(network.subgraph(c).copy() for c in nx.connected_components(network))
net1, net2 = (connected_components[:2])
with open("network.JSON", "w", encoding= 'utf-8') as f:
    # f.write(str(nx.cytoscape_data(net1)))
    json.dump(nx.cytoscape_data(net1), f)

In [28]:
from cdlib import algorithms,viz

communities = algorithms.greedy_modularity(net1, weight= "weight")

experimental_net = nx.set_node_attributes(net1,communities.to_node_community_map(), "group")
net1.nodes["Bigflo"]
    # communities.to_node_community_map()
# pos = nx.spring_layout(net1)
# viz.plot_network_clusters(net1, communities, pos)

{'size': 18, 'group': [12]}

In [15]:
communities.graph